In [ ]:
#Instalando dependencias
#pip install matplotlib
#pip install scipy
#pip install scikit-image
#pip install pillow

In [2]:
!pip install opencv-python

     |████████████████████████████████| 49.9 MB 110 kB/s eta 0:00:01


In [3]:
import matplotlib.cm as cm 
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
import matplotlib
%matplotlib inline

from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import entropy
import scipy.misc
import scipy.ndimage

from skimage import data, io, filters, color, feature
from skimage.color import rgb2gray

from PIL import Image
from PIL import ImageEnhance

from os import listdir
from os.path import isfile, join 
import os

import pywt 
import pywt.data

import math
import csv
import numpy as np
import cv2

print("Librerias cargadas correctamente")

Librerias cargadas correctamente


In [4]:
def graficarImagen(im1, cmap=None): 
    fig = plt.figure(figsize=(16,10))

    ax1 = fig.add_subplot(1,3,1) 

    ax1.set_title('Billete')

    ax1.imshow(im1, cmap=cmap)

In [5]:
#Estas son las partes esenciales que hay que analizar para comprobar si el billete es auténtico o no. Podemos ver que el billete de 100 dólares ha evolucionado (color y forma), lo que hace que muchas de las partes recortadas sean irrelevantes para el análisis.
#Sin embargo, la cabeza del pdte. y el valor del billete (100 dólares) parecen ser estables.

In [6]:
def mejorar_brillo(imagen, brillo):
    brillo_mejorado = ImageEnhance.Brightness(imagen) 
    return brillo_mejorado.enhance(brillo)

def mejorar_nitidez(imagen, nitidez): 
    nitidez_mejorada = ImageEnhance.Sharpness(imagen) 
    return nitidez_mejorada.enhance(nitidez)

def mejorar_color(imagen, color): 
    color_mejorado = ImageEnhance.Color(imagen)
    return color_mejorado.enhance(color)

def mejorar_contraste(imagen, contraste): 
    contraste_mejorado = ImageEnhance.Contrast(imagen) 
    return contraste_mejorado.enhance(contraste)

In [7]:
def graficarHistograma(im1, cmap=None): 
    fig = plt.figure(figsize=(16,4))
    ax1 = fig.add_subplot(1,3,1)
    ax1.set_title('Distribución de grises del billete') 
    ax1.hist(im1.flatten(), bins = 256) 

In [8]:
#En la mayoría de los casos, es posible notar pequeñas diferencias entre los billetes al aplicar los filtros. Sin embargo, este método no es preciso y depende de la subjetividad del observador. Un billete falso muy bueno puede conseguir engañar a los ojos del observador. Además, una persona no puede comparar una gran cantidad de billetes durante un largo periodo de tiempo de forma fiable. Es imposible clasificarlos con el ojo humano.
#Se necesita otro método para clasificar los billetes. El aprendizaje automático puede ayudarnos a resolver este problema.

In [9]:
titles = ['Approximation', ' Horizontal detail', 'Vertical detail', 'Diagonal detail']

def graficarWaveletTrans(coef):
    LL, (LH, HL, HH) = coef
    fig = plt.figure(figsize=(14,6))
    for i, a in enumerate([LL, LH, HL, HH]):
        ax = fig.add_subplot(1,4,i+1)
        ax.imshow(a, interpolation="nearest", cmap='gray') 
        ax.set_title(titles[i], fontsize=15)
        ax.set_xticks([])
        ax.set_yticks([])
        
    fig.tight_layout() 
    plt.show()

In [10]:
def mostrarEstadisticas(billete): 
    print("Varianza:   {} ".format(np.var(billete)))
    print("Oblicuidad: {} ".format(np.mean(skew(billete)))) 
    print("Curtosis:   {} ".format(np.mean(kurtosis(billete))))
    print("Entropía:   {} ".format(np.mean(entropy(billete))))

In [11]:
def normalizar(x): 
    return np.array((x - np.min(x)) / (np.max(x) - np.min(x)))

In [12]:
#NaN, significa "no es un número", es un tipo de dato numérico que se utiliza para representar cualquier valor indefinido o no representable.

In [14]:
#En resumen
def obtenerFeatures(imgUrl):
    billete = Image.open(imgUrl) 
    graficarImagen(billete)
    
    nuevotam = (1250,500)
    billete_redimensionado = billete.resize(nuevotam) 
    graficarImagen(billete_redimensionado)

    izq, arriba, derecha, abajo = 330, 40, 750, 460
    billete_recortado = billete_redimensionado.crop((izq, arriba, derecha, abajo))
    graficarImagen(billete_recortado)

    
    def mejorarImagen():
        imagen_brillo_mejorado = mejorar_brillo(billete_recortado, 1)
        graficarImagen(imagen_brillo_mejorado)

        imagen_nitidez_mejorada = mejorar_nitidez(imagen_brillo_mejorado, 1)
        graficarImagen(imagen_nitidez_mejorada)
        
        imagen_color_mejorado = mejorar_color(imagen_nitidez_mejorada, 1)
        graficarImagen(imagen_color_mejorado)
        
        imagen_contraste_mejorado = mejorar_contraste(imagen_color_mejorado, 1) 
        graficarImagen(imagen_contraste_mejorado)

        imagen_contraste_mejorado.save("images_processed/billete_mejorado.png") 

    def aplicarFiltros():
    
        billete_mejorado = cv2.imread("images_processed/billete_mejorado.png",0)
        
        billete_escalagrises = color.rgb2gray(billete_mejorado)
        billete_escalacolor = color.gray2rgb(billete_escalagrises)

        graficarImagen(billete_escalacolor)
        graficarHistograma(billete_escalacolor)

        plt.imsave("images_processed/billete_escalagrises.png", billete_escalagrises, cmap='gray')
        billete_escalagrises = cv2.imread("images_processed/billete_escalagrises.png",0) 

        billete_canny= cv2.Canny(billete_escalagrises,100,200)
        graficarImagen(billete_canny, 'gray')
        graficarHistograma(billete_canny)

        billete_laplaciano = cv2.Laplacian(billete_escalagrises,cv2.CV_64F) 
        graficarImagen(billete_laplaciano, 'gray')
        graficarHistograma(billete_laplaciano)

        sobel = cv2.Sobel(billete_escalagrises,cv2.CV_64F,0,1,ksize=3) 
        
        billete_sobel = cv2.Sobel(billete_escalagrises,cv2.CV_64F,1,0,ksize=3) 
        graficarImagen(billete_sobel, 'gray')
        graficarHistograma(billete_escalacolor)

        billete_sobel = cv2.Sobel(billete_escalagrises,cv2.CV_64F,1,1,ksize=3) 
        graficarImagen(billete_sobel, 'gray')
        graficarHistograma(billete_sobel)

        billete_gaussiano = cv2.GaussianBlur(billete_escalagrises,(5,5),0) 
        graficarImagen(billete_gaussiano, 'gray')
        graficarHistograma(billete_gaussiano)
        
        billete_bilateral = cv2.bilateralFilter(billete_escalagrises,9,75,75)
        graficarImagen(billete_bilateral, 'gray')
        graficarHistograma(billete_bilateral)


        plt.imsave("images_processed/billete_canny.png", billete_canny, cmap='gray') 

        plt.imsave("images_processed/billete_laplaciano.png", billete_laplaciano, cmap='gray') 

        plt.imsave("images_processed/billete_sobel.png", billete_sobel, cmap='gray') 

        plt.imsave("images_processed/billete_gaussiano.png", billete_gaussiano, cmap='gray') 

        plt.imsave("images_processed/note_bilateral.png", billete_bilateral, cmap='gray') 
    

    def aplicacionFiltroWavelet():
        
        billete_mejorado = cv2.imread("images_processed/billete_mejorado.png") 
        coef = pywt.dwt2(billete_mejorado, 'bior1.3') 
        graficarWaveletTrans(coef)
        plt.imsave("images_processed/billete_wavelet.png", normalizar(coef[1][2])) 

        billete_mejorado = cv2.imread("images_processed/billete_wavelet.png") 
        coef = pywt.dwt2(billete_mejorado, 'bior1.3') 
     
        features= [np.var(coef[1][2]), np.mean(skew(coef[1][2])), np.mean(kurtosis(coef[1][2])), np.mean(entropy(coef[1][2]))]
        for i in range(4):
            if np.isnan(features[i]):
                features[i]=0
        return features

    mejorarImagen()
    aplicarFiltros()
    #guardarModificaciones()
    
    return aplicacionFiltroWavelet()